# 凸最適化としてのロバストMDP

参考

* [On the convex formulations of robust Markov decision processes](https://arxiv.org/abs/2209.10187)

通常の価値反復法は線型計画法のように凸最適化の問題に変形することができます．
凸最適化の形式にすると，
1. 占有率が双対形式として現れる．
2. Sensitivity Analysisが可能
3. 凸最適化の形式は割引率に関して強多項的に解ける唯一の方法

など，いいことが色々あります．しかし，ロバストMDPの場合は凸最適化の形式にするのが難しいです．
実はロバストMDPにKL正則化を入れると凸最適化の問題に変形することができます．

## 表記と準備

* リターン：$R(\pi, \boldsymbol{P})=\mathbb{E}^{\pi, \boldsymbol{P}}\left[\sum_{t=0}^{\infty} \lambda^t r_{S_t A_t} \mid S_0 \sim \boldsymbol{\alpha}\right]$
* 価値関数：$v_s^\pi=\mathbb{E}^{\pi, \boldsymbol{P}}\left[\sum_{t=0}^{\infty} \lambda^t r_{S_t A_t} \mid S_0=s\right]$
* 割引無限ホライゾンマルコフ決定仮定：$(\mathcal{S}, \mathcal{A}, \boldsymbol{r}, \boldsymbol{\alpha}, \lambda, \boldsymbol{P})$
* 不確実集合：$\mathcal{U}=\times_{(s, a) \in \mathcal{S} \times \mathcal{A}} \mathcal{U}_{s a}, \quad \mathcal{U}_{s a} \subseteq \mathbb{R}_{+}^{\mathcal{S}}$

ロバストMDPでは次の問題を解くことを考えます：

$$
\max _{\pi \in \Pi} \min _{\boldsymbol{P} \in \mathcal{U}} R(\pi, \boldsymbol{P})
$$

ロバストベルマン作用素は次で与えられます．

$$
T(\boldsymbol{v})_s=\max _{a \in \mathcal{A}} \min _{\boldsymbol{p} \in \mathcal{U}_{s a}}\left\{r_{s a}+\lambda \cdot \boldsymbol{p}^{\top} \boldsymbol{v}\right\}, \forall s \in \mathcal{S}, \forall \boldsymbol{v} \in \mathbb{R}^{\mathfrak{c}}
$$

このロバストベルマン作用素は$L_\infty$ノルムについて「単調」かつ「縮小性」を満たします（[Robust dynamic programming](http://www.corc.ieor.columbia.edu/reports/techreports/tr-2002-07.pdf)参照．）

---

**縮小性**

$$
\left\|T(\boldsymbol{v})-T\left(\boldsymbol{v}^{\prime}\right)\right\|_{\infty} \leq \lambda \cdot\left\|\boldsymbol{v}-\boldsymbol{v}^{\prime}\right\|_{\infty}, \quad \forall \boldsymbol{v}, \boldsymbol{v}^{\prime} \in \mathbb{R}^{\mathcal{S}}
$$

---

---

**単調性**

$$
v_s \leq v_s^{\prime}, \forall s \in \mathcal{S} \quad \Longrightarrow \quad T(\boldsymbol{v})_s \leq T\left(\boldsymbol{v}^{\prime}\right)_s, \forall s \in \mathcal{S}
$$

---

単調性と縮小性を使えば，次の最適価値の性質を証明できます．

---

**ロバスト作用素の最適価値への収束**

$v^*$をロバストベルマン作用素の唯一の不動点とします．このとき，$v^*$はロバストMDPの最適方策の価値になります．
また，$\pi^*$は決定的かつ$T(v^*)_s$の最大値を与えます．

---



## ロバスト最適作用素と凸最適化問題への変形

凸最適化問題へ変形するために，次のContraction Lemmaが便利です．


---

**Contraction Lemma**

$F: \mathbb{R}^S \to \mathbb{R}^S$を単調な縮小作用素とし，
$g: \mathbb{R}^S \to \mathbb{R}$を非減少な関数とします．
また，$v^\star$を$F$の不動点とします．
このとき，

$$
g\left(\boldsymbol{v}^{\star}\right)=\min \{g(\boldsymbol{v}) \mid \boldsymbol{v} \geq F(\boldsymbol{v})\}=\max \{g(\boldsymbol{v}) \mid \boldsymbol{v} \leq F(\boldsymbol{v})\}
$$

が成り立ちます．
また，$g$が単調増加であれば，$v^\star$は唯一存在します．

証明は[RL_as_LP.ipynb](RL_as_LP.ipynb)に記載しました．

---

$F$にロバストベルマン作用素$T$を適用し，$g(v)=\alpha^\top v$とすれば，Contraction Lemmaから，

$$\boldsymbol{v}^{\star} \in \arg \min \left\{\boldsymbol{\alpha}^{\top} \boldsymbol{v} \mid \boldsymbol{v} \geq T(\boldsymbol{v})\right\}=\arg \max \left\{\boldsymbol{\alpha}^{\top} \boldsymbol{v} \mid \boldsymbol{v} \leq T(\boldsymbol{v})\right\}$$

が成り立ちます．
しかし，これは凸最適化問題ではありません．

---

**凸最適化問題**

凸関数$f_0, \cdots, f_m$が凸関数のとき，次の問題を凸最適化問題といいます：

$$
\begin{array}{ll}
\operatorname{minimize} & f_0(x) \\
\text { subject to } & f_i(x) \leq 0, \quad i=1, \ldots, m \\
& a_i^T x=b_i, \quad i=1, \ldots, p
\end{array}
$$

---

普通のベルマン作用素の場合は凸最適化問題になります．
ベルマン作用素を思い出すと，$T_{\boldsymbol{P}}(\boldsymbol{v})_s=\max _{a \in \mathcal{A}} r_{s a}+\lambda \boldsymbol{P}_{s a}^{\top} \boldsymbol{v}, \quad \forall s \in \mathcal{S}, \forall \boldsymbol{v} \in \mathbb{R}^{\mathcal{S}}$は線形関数にMaximumを適用した形です．
ここで，[Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)より，MaximumはConvexを保存するので，普通のベルマン作用素を使った定式化は凸最適化問題です．

一方で，ロバストベルマン作用素は
$T(\boldsymbol{v})_s=\max _{a \in \mathcal{A}} \min _{\boldsymbol{p} \in \mathcal{U}_{s a}}\left\{r_{s a}+\lambda \cdot \boldsymbol{p}^{\top} \boldsymbol{v}\right\}, \forall s \in \mathcal{S}, \forall \boldsymbol{v} \in \mathbb{R}^{\mathfrak{c}}$なので，$\max$と$\min$が出てきてしまい，ConvexでもConcaveでもありません．

また，楽観的なベルマン作用素は$\max$が連続で続くのでConvexです．


## ロバスト正則化ベルマン作用素

適当な方策$\nu$を使って，次のようなロバスト正則化ベルマン作用素を考えます．

$$
\tilde{T}(\boldsymbol{v})_s=\max _{\boldsymbol{\pi}_s \in \Delta(\mathcal{A})} \sum_{a \in \mathcal{A}} \pi_{s a}\left(r_{s a}+\lambda \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \boldsymbol{p}^{\top} \boldsymbol{v}\right)-\frac{1}{b} \cdot \mathrm{KL}\left(\boldsymbol{\pi}_s, \boldsymbol{\nu}_s\right), \forall s \in \mathcal{S}
$$

ここで，$\mathrm{KL}\left(\boldsymbol{\pi}_s, \boldsymbol{\nu}_s\right)=\sum_{a \in \mathcal{A}} \pi_{s a} \log \left(\pi_{s a} / \nu_{s a}\right)$です．

まず，このようなロバスト正則化ベルマン作用素に対して次が成立します．

---

**ロバスト正則化ベルマン作用素の性質**

1. $\tilde{T}$は$L_\infty$ノルムについて単調縮小性を持ちます．証明はAppendix Bにまかせます
2. ロバストベルマン作用素との差：$\tilde{T}(\boldsymbol{v})_s \leq T(\boldsymbol{v})_s \leq \tilde{T}(\boldsymbol{v})_s+\frac{\log (|\mathcal{A}|)}{b}, \quad \forall s \in \mathcal{S}, \forall \boldsymbol{v} \in \mathbb{R}^{\mathcal{S}}$
3. 最適価値関数との差：$\tilde{v}_s^{\star} \leq v_s^{\star} \leq \tilde{v}_s^{\star}+\frac{\log (|\mathcal{A}|)}{b \cdot(1-\lambda)}, \quad \forall s \in \mathcal{S}$

---

KL正則化について，次のような変形が成立します

---

**Donsker Varadhan variational formula**

$\boldsymbol{q}_0 \in \Delta(\mathcal{A})$と$a\in \mathcal{A}$について$q_{0, a}>0$，そして$\boldsymbol{y} \in \mathbb{R}^{\mathcal{A}}$について，次が成立します．

$$
\max _{\boldsymbol{q} \in \Delta(\mathcal{A})} \boldsymbol{q}^{\top} \boldsymbol{y}-\frac{1}{b} \mathrm{KL}\left(\boldsymbol{q}, \boldsymbol{q}_0\right)=\frac{1}{b} \log \left(\sum_{a \in \mathcal{A}} q_{0, a} \exp \left(b \cdot y_a\right)\right)
$$

---

これを使うと，ロバスト正則化ベルマン作用素は

$$
\tilde{T}(\boldsymbol{v})_s=\frac{1}{b} \log \left(\sum_{a \in \mathcal{A}} \nu_{s a} \exp \left(b \cdot r_{s a}+b \cdot \lambda \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \boldsymbol{p}^{\top} \boldsymbol{v}\right)\right)
$$

の形に変形されます．
また，このロバスト正則化ベルマン作用素は単調縮小性を満たし，かつその唯一の不動点が$\tilde{v}^*$なので，

$$
\max \{g(\boldsymbol{v}) \mid \boldsymbol{v} \leq \tilde{T}(\boldsymbol{v})\}
$$

を解くと，Contraction Lemmaから，これは$\tilde{v}^*$を求めます．
しかし，$\boldsymbol{v} \mapsto \tilde{T}(\boldsymbol{v})$が凸でも凹でもないので，これはまだ凸最適化問題になっていません．


## 指数変数による変換

$\boldsymbol{v} \mapsto \tilde{T}(\boldsymbol{v})_s$は凸でも凹でもないので，$\left\{\boldsymbol{v} \in \mathbb{R}^{\mathcal{S}} \mid \boldsymbol{v} \leq \tilde{T}(\boldsymbol{v})\right\}$が凸でない可能性があります．
この問題を変数を変換して凸にしましょう．

まず，次で定義された関数を考えます．

$$
\log _b(z)=\log (z) / b, \quad \exp _b\left(z^{\prime}\right)=\exp \left(b \cdot z^{\prime}\right), \quad \forall z^{\prime}>0, \forall z \in \mathbb{R} .
$$

これを使うと，ロバスト正則化ベルマン作用素は次の形に変形できます．

$$
\begin{aligned}
\tilde{T}(\boldsymbol{v})_s & =\log _b\left(\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) \exp _b\left(\lambda \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \boldsymbol{p}^{\top} \boldsymbol{v}\right)\right) \\
& =\log _b\left(\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \exp _b\left(\lambda \boldsymbol{p}^{\top} \boldsymbol{v}\right)\right) \\
& =\log _b\left(\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \prod_{s^{\prime} \in \mathcal{S}}\left(\exp _b\left(v_{s^{\prime}}\right)\right)^{\lambda p_{s^{\prime}}}\right)
\end{aligned}
$$

ここで，次の$\tilde{t}:\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}} \rightarrow\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}}$なる作用素を定義しましょう．

$$
\tilde{t}(\boldsymbol{x})_s=\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \prod_{s^{\prime} \in \mathcal{S}} x_{s^{\prime}}^{\lambda p_{s^{\prime}}}
$$

これを使うと，ロバスト正則化ベルマン作用素は次の形に変形できます．

$$
\tilde{t}(\boldsymbol{x})_s=\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) \min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \prod_{s^{\prime} \in \mathcal{S}} x_{s^{\prime}}^{\lambda p_{s^{\prime}}}
$$

ここで，$\mathbf{x}\mapsto \tilde{t}(\mathbf{x})_s$は凹関数であることを証明しましょう．

---

それぞれの$s\in \mathcal{S}$について，$\mathbf{x}\mapsto \tilde{t}(\mathbf{x})_s$は$\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}}$について凹関数．

証明しましょう．まず$s$を固定します．

$\tilde{t}$をよくみると，$\sum_{a \in \mathcal{A}} \nu_{s a} \exp _b\left(r_{s a}\right) $の部分は$x$に関係ない重みつき和です．
[Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)の3.2.1より，凹関数の重み付き和も凹関数です．
よって，$\min _{\boldsymbol{p} \in \mathcal{U}_{s a}} \prod_{s^{\prime} \in \mathcal{S}} x_{s^{\prime}}^{\lambda p_{s^{\prime}}}$が凹関数であることを示せばいいですが，
[Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)の3.2.2より，$\min$をとっても凹関数は保存されます．よって，
$$\mathbf{x}\mapsto \prod_{s^{\prime} \in \mathcal{S}} x_{s^{\prime}}^{\lambda p_{s^{\prime}}}$$
が凹関数であることを示せば大丈夫です．
ここで関数$x\mapsto x^{\lambda p_s'}$は$\lambda p_s'\in [0, 1)$なので単調かつ凹関数です．
これと[A convex programming-based algorithm for mean payoff stochastic games with perfect information](https://link.springer.com/article/10.1007/s11590-017-1140-y)のFact2をつかうと，$\mathbf{x}\mapsto \prod_{s^{\prime} \in \mathcal{S}} x_{s^{\prime}}^{\lambda p_{s^{\prime}}}$が凹関数であることが示せます．

---


## 凸最適化問題への変形

これまでで，ロバスト正則化ベルマン作用素：$\tilde{T}=\log _b \circ \tilde{t} \circ \exp _b$を定義し，その$\tilde{t}$が入力ベクトルの各要素について凹関数であることを見てきました．
ここで，$\log _b:\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}} \rightarrow \mathbb{R}^{\mathcal{S}}$と$\exp _b: \mathbb{R}^{\mathcal{S}} \rightarrow\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}}$は一対一対応のマッピングなので，次の変数変換を考えます．

$$
\boldsymbol{x} \in\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}}, x_s=\exp _b\left(v_s\right), \forall s \in \mathcal{S} .
$$

これをつかうと，

$$
\boldsymbol{v} \leq \tilde{T}(\boldsymbol{v}) \quad \Longleftrightarrow \quad \boldsymbol{v} \leq \log _b \circ \tilde{t} \circ \exp _b(\boldsymbol{v}) \quad \Longleftrightarrow \quad \exp _b(\boldsymbol{v}) \leq \tilde{t} \circ \exp _b(\boldsymbol{v}) \quad \Longleftrightarrow \quad \boldsymbol{x} \leq \tilde{t}(\boldsymbol{x})
$$

が得られ，$\left\{\boldsymbol{x} \in\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}} \mid \boldsymbol{x} \leq \tilde{t}(\boldsymbol{x})\right\}$は凸集合になります．

以上を踏まえると，次の定理が成り立ちます．

---

$\tilde{v}^*$をロバスト正則化ベルマン作用素の不動点とします．このとき，$\tilde{v}^*$は次の（非凸かもしれない）問題の最適解になります．

$$
\max \left\{\sum_{s \in \mathcal{S}} \exp _b\left(v_s\right) \mid \boldsymbol{v} \leq \tilde{T}(\boldsymbol{v}), \boldsymbol{v} \in \mathbb{R}^{\mathcal{S}}\right\}
$$

ここで，$\tilde{\boldsymbol{x}}^{\star} \in \mathbb{R}^{\mathcal{S}}$に対して$\tilde{\boldsymbol{v}}^{\star}=\log _b\left(\tilde{\boldsymbol{x}}^{\star}\right)$なる変数変換を考えます．このとき，$\tilde{\boldsymbol{x}}^{\star} \in \mathbb{R}^{\mathcal{S}}$は次の凸最適化問題の最適解になります．

$$
\max \left\{\sum_{s \in \mathcal{S}} x_s \mid \boldsymbol{x} \leq \tilde{t}(\boldsymbol{x}), x_s \geq 1, \forall s \in \mathcal{S}\right\}
$$

証明します．まず最初のやつはContraction Lemmaから成立します．
続いて２つ目の最適化が元の問題に一致するのも今までの議論から明らかです．

最後に，途中の変数変換で$\boldsymbol{x} \in\left(\mathbb{R}_{+}^*\right)^{\mathcal{S}}$が出てきましたが，これは不等号を含んでいるため，これについて丁寧に議論しないといけません．
報酬が非負であることを過程すれば，$\tilde{\boldsymbol{v}}^{\star} \geq \mathbf{0}$なので，最初の問題での実行可能解を

$$\left\{\boldsymbol{v} \in \mathbb{R}^{\mathcal{S}} \mid \boldsymbol{v} \leq \tilde{T}(\boldsymbol{v}), \boldsymbol{v} \geq \mathbf{0}\right\}$$

に限定しても問題ありません．これを変数変換すると，$\left\{\boldsymbol{x} \in \mathbb{R}^{\mathcal{S}} \mid \boldsymbol{x} \leq \tilde{t}(\boldsymbol{x}), x_s \geq 1, \forall s \in \mathcal{S}\right\}$であり，これは閉集合です．

---

---

**変数変換した問題の解の精度**

上で得られた凸最適化問題を解くと$\tilde{v}^*$が得られます．これは$v^*$ではありません．

$\epsilon>0$ および $b \geq \frac{\log (|\mathcal{A}|)}{\epsilon(1-\lambda)}$として，$\tilde{\boldsymbol{v}}^{\star}=\log _b\left(\tilde{\boldsymbol{x}}^{\star}\right)$とします．このとき，

$$
\tilde{v}_s^{\star} \leq v_s^{\star} \leq \tilde{v}_s^{\star}+\epsilon
$$

が成立します．

---